In [1]:
import numpy as np
import scipy
import sympy as sp

import plotly.graph_objects as go
from qiskit.opflow import I, X, Y, Z, H

from qaoa import *

import qiskit.tools.jupyter
%qiskit_version_table

import pickle
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
def graph_energy(res, energy, points = None):
    s = np.linspace(0, 2 * np.pi, res + 1)[:-1]
    
    fig = go.Figure(data=[
            go.Surface(x=s, y=s, z=energy, showscale=False,
                       colorscale='Viridis')])

    
    if points is not None:
        px, py, p_energy = points
        fig.add_trace(go.Scatter3d(x=px, y=py, z=p_energy,
                                   mode='markers'))

    fig.update_scenes(
        xaxis_title='Gamma',
        yaxis_title='Beta',
        zaxis_title='Energy'
    )

    fig.update_layout(title='p = 1 Energy Landscape',
                    autosize=False,
                    width=1000, height=700,
                    )
    fig.show()

In [3]:
"""
Schwinger
"""
select = {'N': 4, #qubits
          'g' : 1,  #coupling
          'm' : 1,  #bare mass
          'a' : 1, #lattice spacing
          'theta' : 0, #topological term
          'mixer_type' : 'Y', # type of mixer {'X', 'Y', 'XY'}
         }

model = QuantumModel('schwinger', 'Standard')


ham = model.make(select)
schwinger_target = ham.target
ham

driver:
4.0 * IIII
+ 2.0 * IIZZ
+ 1.0 * IZIZ
+ 1.0 * IZZI
+ 4.0 * IIIZ
- 1.0 * IIZI
+ 3.0 * IZII
- 2.0 * ZIII
mixer:
1.0 * IIIY
+ 1.0 * IIYI
+ 1.0 * IYII
+ 1.0 * YIII
target:
1.0 * IIII
+ 0.5 * IIZZ
+ 0.25 * IZIZ
+ 0.25 * IZZI
+ 1.0 * IIIZ
- 0.25 * IIZI
+ 0.75 * IZII
- 0.5 * ZIII
+ 0.25 * IIIY
+ 0.25 * IIYI
+ 0.25 * IYII
+ 0.25 * YIII

In [4]:
def find_landscape_minimas(landscape, filter = True):
    less_right = landscape <= np.roll(landscape, 1 , 0)
    less_left = landscape <= np.roll(landscape, -1 , 0)
    less_gamma = np.logical_and(less_right, less_left)

    less_bottom = landscape <= np.roll(landscape, 1 , 1)
    less_top = landscape <= np.roll(landscape, -1 , 1)
    less_beta = np.logical_and(less_bottom, less_top)

    minimas = np.logical_and(less_gamma, less_beta)

    res = landscape.shape[0]
    s = np.linspace(0, 2 * np.pi, res + 1)[:-1]

    x, y = np.meshgrid([s], [s])
    min_x = x[minimas]
    min_y = y[minimas]
    min_energy = landscape[minimas]

    if filter:
        filtered = min_energy <= np.mean(min_energy)
        min_x = min_x[filtered]
        min_y = min_y[filtered]
        min_energy = min_energy[filtered]

    return np.vstack([min_x, min_y, min_energy])

In [5]:
class ParametersNode:
    def __init__(self, 
                 parameters,
                 energy):
        
        self.parameters = parameters
        self.energy = energy

        self.id = tuple(np.round(parameters, 4))

        self.p = len(parameters) // 2

        self.gammas = parameters[::2]
        self.betas = parameters[1::2]

        self.layer = None

        self.prev = {}
        self.next = {}

    def optimize_next(self, 
                      energy_fn, 
                      initial_parameters):
        
        next_p = len(initial_parameters) // 2
        
        next_min = scipy.optimize.minimize(
                                    fun = lambda x: energy_fn(*x), 
                                    x0 = initial_parameters,
                                    bounds = [[0, 2 * np.pi] for _ in range(next_p * 2)],
                                    tol = 1e-6)
        
        next_parameters = next_min.x
        next_energy = next_min.fun
        next_node = ParametersNode(next_parameters,
                                   next_energy)
        
        return next_node

class NodeLayer:
    def __init__(self):
        self.nodes = {}
        self.num_nodes = 0

        self.prev = None
        self.next = None

    def optimize_next(self, energy_fn, parameter_fn):
        for node in self.nodes.values():
            initial_parameters = parameter_fn(node.parameters)
            
            new_node = node.optimize_next(energy_fn, initial_parameters)

            if self.next is None:
                self.next = NodeLayer()
                self.next.prev = self

            # if parameter already exists, throw away current node and reference to originally created node
            if new_node.id in self.next.nodes.keys():
                new_node = self.next.nodes[new_node.id]
            else:
                self.next.add_node(new_node)

            node.next[new_node.id] = new_node
            new_node.prev[node.id] = node

    def add_node(self, new_node: ParametersNode):
        self.nodes[new_node.id] = new_node
        self.num_nodes = len(self.nodes)

In [6]:
def find_minimas(hamiltonian, 
                 verbose = True):
    
    start = time.time()
    layers = []

    ### Find p = 1 minimas ###

    symqaoa = QAOA_Sympy(ham, p=1)

    if verbose: print(f"Making p = 1 Expectation {time.time() - start}")
    symqaoa.make()

    if verbose: print(f"Making p = 1 Landscape {time.time() - start}")
    sym_energies = symqaoa.energy_landscape(res=128)

    energy_minimas = find_landscape_minimas(sym_energies)

    energy_min_x, energy_min_y, min_energies = energy_minimas

    minimas = []

    for x, y in zip(energy_min_x, energy_min_y):
        minimas.append(scipy.optimize.minimize(fun = lambda x: symqaoa.energy(*x), 
                                x0 = [x, y],
                                bounds = [[0, 2 * np.pi] for _ in range(2)]))
        
    avg_energy = 0
    for minima in minimas:
        avg_energy += minima.fun 
    avg_energy /= len(minimas)

    minimas = [minima for minima in minimas if minima.fun < avg_energy]

    layers.append(NodeLayer())


    if verbose:print("\np = 1 energies:")
    for minima in minimas:
        if verbose: print(f"{minima.fun}")
        node = ParametersNode(minima.x, minima.fun)
        layers[0].add_node(node)
    if verbose:print("")

    ### Find p = 2 minimas ###

    if verbose: print(f"Making p = 2 Expectation {time.time() - start}")
    symqaoa.add_round()
    symqaoa.make_expectation()
    
    def perturb_params(perturbation):
        def parameter_fn(parameters):
            new_parameters = np.hstack([parameters, parameters])
            return new_parameters + perturbation
        return parameter_fn
    
    if verbose: print(f"Optimizing p = 2 {time.time() - start}")
    magnitudes = [0.2, 1]
    for magnitude in magnitudes:
        perturbations = [0, magnitude, -magnitude]
        for pgamma in perturbations:
            for pbeta in perturbations:
                parameter_fn = perturb_params(np.array([0, 0, pgamma, pbeta]))
                layers[0].optimize_next(symqaoa.energy, parameter_fn)

    layers.append(NodeLayer())

    prior_min_energy = min([node.energy for node in layers[0].nodes.values()])

    if verbose: print("\np = 2 energies:")
    for node in layers[0].next.nodes.values():
        if node.energy <= prior_min_energy:
            if verbose: print(node.energy)
            layers[1].add_node(node)
    if verbose: print("")

    ### Find p >= 3 minimas ###

    def interpolate_params(parameters):
        p = len(parameters) // 2
        gammas, betas = parameters[::2], parameters[1::2]

        x_fit = np.linspace(0, 1, p)
        x_poly = np.linspace(0, 1, p + 1)

        gamma_interp = np.interp(x_poly, x_fit, gammas)
        beta_interp = np.interp(x_poly, x_fit, betas)
        new_parameters = np.array([val for pair in zip(gamma_interp, beta_interp) for val in pair])
        return new_parameters

    def fit_params(parameters):
        p = len(parameters) // 2
        gammas, betas = parameters[::2], parameters[1::2]

        x_fit = np.linspace(0, 1, p)
        x_poly = np.linspace(0, 1, p + 1)

        gamma_fit = np.poly1d(np.polyfit(x_fit, gammas, 2))
        beta_fit = np.poly1d(np.polyfit(x_fit, betas, 2))

        new_parameters = np.array([val for pair in zip(gamma_fit(x_poly), beta_fit(x_poly)) for val in pair])
        return new_parameters
    
    numqaoa = QAOA_Numpy(ham)
    
    for i in range(1, 4):
        layers[i].optimize_next(numqaoa.energy, fit_params)
        layers[i].optimize_next(numqaoa.energy, interpolate_params)

        layers.append(NodeLayer())

        prior_min_energy = min([node.energy for node in layers[i].nodes.values()])

        if verbose: print(f"\np = {i + 1} energies:")
        for node in layers[i].next.nodes.values():
            if node.energy <= prior_min_energy:
                if verbose: print(node.energy)
                layers[i + 1].add_node(node)
        if verbose: print("")

    return layers

In [7]:
layers = find_minimas(ham)

Making p = 1 Expectation 0.6083781719207764
Making p = 1 Landscape 11.757657051086426

p = 1 energies:
-2.07964928701366
-2.0796492870136576
-2.0796492870136616
-2.0796492870136487

Making p = 2 Expectation 20.8154559135437
Optimizing p = 2 134.863135099411

p = 2 energies:
-2.081283546201762
-2.081283541773093
-2.0803956734008016
-2.0864960522815146
-2.113208029285451
-2.113208128064059
-2.1132081279380857
-2.1132081204032422
-2.113208127701066
-2.091840212791278
-2.109174241721617
-2.1091744238847534
-2.1091744056193855
-2.109174404070279
-2.0797336917617453
-2.0797336917621614



In [18]:
layers

In [55]:
def interpolate_params(parameters):
        p = len(parameters) // 2
        gammas, betas = parameters[::2], parameters[1::2]

        x_fit = np.linspace(0, 1, p)
        x_poly = np.linspace(0, 1, p + 1)

        gamma_interp = np.interp(x_poly, x_fit, gammas)
        beta_interp = np.interp(x_poly, x_fit, betas)
        new_parameters = np.array([val for pair in zip(gamma_interp, beta_interp) for val in pair])
        return new_parameters

def fit_params(parameters):
    p = len(parameters) // 2
    gammas, betas = parameters[::2], parameters[1::2]

    x_fit = np.linspace(0, 1, p)
    x_poly = np.linspace(0, 1, p + 1)

    gamma_fit = np.poly1d(np.polyfit(x_fit, gammas, 2))
    beta_fit = np.poly1d(np.polyfit(x_fit, betas, 2))

    new_parameters = np.array([val for pair in zip(gamma_fit(x_poly), beta_fit(x_poly)) for val in pair])
    return new_parameters

numqaoa = QAOA_Numpy(ham)

verbose = True

for i in range(5, 20):
    if verbose: print(f"Fit p = {i + 2}")
    layers[i].optimize_next(numqaoa.energy, fit_params)
    if verbose: print(f"Interpolate p = {i + 2}")
    layers[i].optimize_next(numqaoa.energy, interpolate_params)

    layers.append(NodeLayer())

    prior_min_energy = min([node.energy for node in layers[i].nodes.values()])

    if verbose: print(f"\np = {i + 1} energies:")
    for node in layers[i].next.nodes.values():
        if node.energy < prior_min_energy:
            if verbose: print(node.energy)
            layers[i + 1].add_node(node)
    if verbose: print("")

    print(min([node.energy for node in layers[i].next.nodes.values()]))

    with open('data', 'wb') as f:
        pickle.dump(layers,f)

Fit p = 7
Interpolate p = 7

p = 6 energies:
-2.1558553599335792
-2.155887944159434
-2.1558883688381383
-2.1558906597753302
-2.1558827654058907
-2.1558695793653473
-2.1558549261576854
-2.1559030225913727
-2.155851671448172
-2.1559501814455233

-2.1559501814455233
Fit p = 8
Interpolate p = 8

p = 7 energies:
-2.1559656974499584

-2.1559656974499584
Fit p = 9
Interpolate p = 9

p = 8 energies:

-2.155857707711967
Fit p = 10
Interpolate p = 10


ValueError: min() arg is an empty sequence

In [104]:
converged_parameters = []
converged_energies = []

def find_converged_parameters(node):
    converged_energies.append(node.energy)
    converged_parameters.append(node.parameters)
    if node.p > 1:
        for node in node.prev.values():
            find_converged_parameters(node)
            

for i, node in enumerate(layers[7].next.nodes.values()):
        if i == 1:
            find_converged_parameters(node)

In [105]:
converged_energies

[-2.155857707711967,
 -2.1559656974499584,
 -2.1558553599335792,
 -2.155522553277559,
 -2.154824109487455,
 -2.1309447572121893,
 -2.120272801847749,
 -2.109174241721617,
 -2.07964928701366]

In [106]:
def closeness_to_ground(val):
    return 1 - (ham.eigen_values()[-1] - val) / (ham.eigen_values()[-1] - ham.eigen_values()[0])

closeness_to_ground(np.array(converged_energies)) * 100

array([1.87232156e-03, 1.63165820e-04, 1.90947989e-03, 7.17681748e-03,
       1.82311013e-02, 3.96170086e-01, 5.65075337e-01, 7.40732470e-01,
       1.20802452e+00])

In [33]:
ham.eigen_values()[0]

-2.1559760067703464

In [131]:
import plotly.graph_objects as go

def plot_parameters(converged_parameters):
    fig = go.Figure()

    num_lines = len(converged_parameters)

    for i, parameters in enumerate(converged_parameters[::-1]):
        p = len(parameters) // 2

        gammas = parameters[::2]
        betas = parameters[1::2]

        print(p)
        
        fig.add_trace(go.Scatter(x=np.linspace(0, 1, p), y = gammas,
                                 opacity= ((i + 1) / num_lines) ** 5,
                                 mode="lines",
                                 line=dict(color='red')))
        fig.add_trace(go.Scatter(x=np.linspace(0, 1, p), y = betas,
                                 opacity= ((i + 1) / num_lines) ** 5,
                                 mode="lines",
                                 line=dict(color='blue')))
    fig.update_layout(showlegend=False, 
                    width=1200,
                    height=700,
                    title = "Parameter Curves",
                    xaxis_title = "Step (i-1)/(p-1)",
                    yaxis_title = "Gamma(red), Beta(blue)")
            
    fig.show()
    
plot_parameters(converged_parameters)

1
2
3
4
5
6
7
8
9


In [138]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1, 1 +len(converged_energies))), y = (1 - closeness_to_ground(np.array(converged_energies[1::][::-1]))) * 100,
                                 line=dict(color='violet')))

fig.update_layout(showlegend=False,
                  width=1200,
                    height=700,
                  title = "Performance Scaling of p",
                  xaxis_title = "p - number of rounds",
                  yaxis_title = "Closeness to Ground (%)")

fig.show()

In [139]:
(1 - closeness_to_ground(np.array(converged_energies[1::][::-1]))) * 100

array([98.79197548, 99.25926753, 99.43492466, 99.60382991, 99.9817689 ,
       99.99282318, 99.99809052, 99.99983683])

In [137]:
closeness_to_ground(np.array(converged_energies[1::][::-1]))[-1]

1.6316581957775966e-06

In [103]:
def closeness_to_ground(val):
    return 1 - (ham.eigen_values()[-1] - val) / (ham.eigen_values()[-1] - ham.eigen_values()[0])

closeness_to_ground(-2.1442988) * 100

0.18481538464456548

In [21]:
with open('data', 'wb') as f:
    pickle.dump(layers,f)

with open('data', 'rb') as f:
    y = pickle.load(f)

In [144]:
numqaoa.energy(*converged_parameters[0])

-2.155857707711967

In [158]:
numqaoa.energy(*(np.random.rand(18) * 2 * np.pi))

0.5813156838282922

In [159]:
scipy.optimize.minimize(
                        fun = lambda x: numqaoa.energy(*x), 
                        x0 = np.random.rand(18) * 2 * np.pi,
                        bounds = [[0, 2 * np.pi] for _ in range(9 * 2)],
                        tol = 1e-6)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: -2.1389073211120793
        x: [ 0.000e+00  5.622e+00 ...  6.566e-01  2.418e+00]
      nit: 111
      jac: [ 4.540e-01  1.935e-03 ... -9.271e-03 -5.471e-03]
     nfev: 2451
     njev: 129
 hess_inv: <18x18 LbfgsInvHessProduct with dtype=float64>